In [1]:
import numpy as np
from scipy.constants import hbar,h,e,c
F0=2.0678e-15
def I(f,Ec):
    Ej=Ejf(f,Ec)
    I=Ej*2*np.pi/F0*h*1e18
    return I
def Ejf(f,Ec):
    Ej=(f+Ec)**2/(8*Ec)
    return Ej
def Eji(I):
    return I*F0/(2*np.pi)/1e9/h/1e9
def f(Ej,Ec):
    return np.sqrt(8*Ej*Ec)-Ec

In [54]:
e_eff=(11.45+1)/2## for all calculations

## Resonator properties

In [55]:
Mhz=(2*np.pi*1e6)

In [56]:
fr=6.5e9
Q=2000
k=np.pi*fr/Q #amplitude
k/Mhz

1.625

In [39]:
Cclaw=52e-3#pF
Cl_r=163.4#pF/m
Leff_add=Cclaw/Cl_r
Leff_add*1e3

0.3182374541003672

In [45]:
fq=4.5e9

In [57]:
Zr=50
Ccr=3.1125e-15
Cq=79e-15
Cr=1/(2*np.pi*fr*Zr)
g=2*np.pi*Ccr/np.sqrt(Cr*Cq)*np.sqrt(fr*fq)
g/Mhz

85.58356326398496

In [58]:
hi=g**2/(2*np.pi*(fr-fq))
hi/Mhz

3.662273150480256

## Qubit Frequencies

In [3]:
j=0.5e-3

In [29]:
f1=4.5
f2=4.7
fmin=4
fmax=10
Ec=0.265

In [30]:
Ejf(f1,Ec),Ejf(f2,Ec)

(10.710011792452828, 11.627936320754717)

In [31]:
(I(fmin,Ec)+I(fmax,Ec))/2,(I(fmax,Ec)-I(fmin,Ec))/2

(58.673363375511144, 41.3979053902528)

In [32]:
(I(fmin,Ec)+I(fmax,Ec))/2/j/300,(I(fmax,Ec)-I(fmin,Ec))/2/j//300

(391.155755836741, 275.0)

In [33]:
I(f1,Ec)/j/200,I(f2,Ec)/j/200

(215.6340345066305, 234.1154118613981)

In [34]:
Sl=200*215
Sr=200*234
S2=300*391
S1=300*275

In [35]:
S2/S1

1.4218181818181819

In [36]:
Il=j*Sl
Ir=j*Sr
I1=j*S1
I2=j*S2
Il,Ir,I1,I2

(21.5, 23.400000000000002, 41.25, 58.65)

## For two-qubit gates

In [ ]:
w_1=f
w_2=6.51e3
w_c=7.026e3
g=70
J=(g**2/2)*(1/(w_c-w_1)+1/(w_c-w_2))
delta=0.265*1e3
delta12=np.abs(w_2-w_1)
zz=-2*(2*delta)/((delta12-delta)*(delta12+delta))*J**2
zx = J/delta12*delta/(delta+delta12)
def coupl(w_c):
  w_1=7.01e3
  w_2=6.516e3
  g=70
  J=(g**2/2)*(1/(w_c-w_1)+1/(w_c-w_2))
  delta=0.265*1e3
  delta12=np.abs(w_2-w_1)
  zz=-2*(2*delta)/((delta12-delta)*(delta12+delta))*J**2
  zx = J/delta12*delta/(delta+delta12)
  return J,zz,zx
J,zx,zz